# 아마존 세이지메이커 점프스타트에서 Llama 2 모델 미세 조정하기

In [ ]:
%pip install -U sagemaker==2.202.1 datasets==2.15.0

## 사전 훈련된 모델 배포

---

먼저 Llama 2 모델을 세이지메이커 엔드포인트로 베포합니다.

---

In [3]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In [22]:
from sagemaker.jumpstart.model import JumpStartModel

pretrained_model = JumpStartModel(model_id=model_id, model_version=model_version)
pretrained_predictor = pretrained_model.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-2-7b-2024-01-01-22-18-03-262
INFO:sagemaker:Creating endpoint-config with name meta-textgeneration-llama-2-7b-2024-01-01-22-18-03-340
INFO:sagemaker:Creating endpoint with name meta-textgeneration-llama-2-7b-2024-01-01-22-18-03-340


--------------!

## 미세 조정을 위한 데이터 세트 준비

---

데이터 세트를 도메인 적응 형식 또는 명령어 조정 형식으로 미세 조정할 수 있습니다. 자세한 내용은 [데이터 세트 지침](#Dataset-instruction) 섹션을 참조하세요. 이 실습에서는 명령어 조정 형식의 Dolly 데이터 세트 일부를 사용합니다. [Dolly 데이터 세트](https://huggingface.co/datasets/databricks/databricks-dolly-15k)는 질문 응답, 요약, 정보 추출 등 다양한 범주에 대해 약 15,000개의 명령어-응답 항목을 포함하고 있으며, Apache 2.0 라이선스 하에 제공됩니다. 우리는 미세조정을 위해 요약 예제를 선택할 것입니다.

훈련 데이터는 각 줄이 하나의 데이터 샘플을 나타내는 딕셔너리인 JSON 라인 (.jsonl) 형식으로 되어 있습니다. 모든 훈련 데이터는 하나의 폴더에 있어야 하지만, 여러 개의 jsonl 파일에 저장할 수 있습니다. 또한, 입력과 출력 형식을 설명하는 template.json 파일을 훈련 폴더에 포함할 수 있습니다.

비정형 데이터 세트(텍스트 파일 모음)으로 모델을 훈련시키려면 부록의 [도메인 적응 데이터 세트 형식으로 미세조정 예제](#Example-fine-tuning-with-Domain-Adaptation-dataset-format) 섹션을 참조하세요.

---

In [5]:
from datasets import load_dataset

dolly_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# 질문 응답이나 정보 추출을 위해 훈련하려면 다음 줄에서 예제["category"] == "closed_qa"/"information_extraction"으로 조건문을 변경할 수 있습니다.
summarization_dataset = dolly_dataset.filter(lambda example: example["category"] == "summarization")
summarization_dataset = summarization_dataset.remove_columns("category")

# 데이터 세트를 두 개로 분리하여 테스트 데이터는 마지막 모델 평가에 사용합니다.
train_and_test_dataset = summarization_dataset.train_test_split(test_size=0.1)
train_and_test_dataset["test"][0]

{'instruction': 'Please describe  what is oil and give me a list of it’s applications.',
 'context': 'An oil is any nonpolar chemical substance that is composed primarily of hydrocarbons and is hydrophobic (does not mix with water) & lipophilic (mixes with other oils). Oils are usually flammable and surface active. Most oils are unsaturated lipids that are liquid at room temperature.\n\nThe general definition of oil includes classes of chemical compounds that may be otherwise unrelated in structure, properties, and uses. Oils may be animal, vegetable, or petrochemical in origin, and may be volatile or non-volatile. They are used for food (e.g., olive oil), fuel (e.g., heating oil), medical purposes (e.g., mineral oil), lubrication (e.g. motor oil), and the manufacture of many types of paints, plastics, and other materials. Specially prepared oils are used in some religious ceremonies and rituals as purifying agents.',
 'response': 'An oil is a chemical substance that is composed primar

## 엔드포인트 호출하기

---

다음으로 몇 가지 샘플 쿼리를 사용하여 엔드포인트를 호출합니다. 이후에 이 노트북에서는 해당 모델을 사용자 정의 데이터 세트로 미세 조정하고, 미세 조정된 모델을 사용하여 추론을 수행합니다. 또한 사전 훈련된 모델과 미세 조정된 모델을 통해 얻은 결과를 비교합니다.

---

In [6]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [7]:
test_dataset = train_and_test_dataset["test"]

inputs, ground_truth_responses, responses_before_finetuning, responses_after_finetuning = (
    [],
    [],
    [],
    [],
)

def predict_and_print(datapoint):
    # 명령어 기반 미세 조정을 위해 입력과 출력 사이에 특별한 키를 삽입합니다.
    input_output_demarkation_key = "\n\n### Response:\n"

    prompt = f'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{datapoint["instruction"]}\n\n### Input:\n{datapoint["context"]}\n\n',
    
    payload = {
        "inputs": prompt[0] + input_output_demarkation_key,
        "parameters": {"max_new_tokens": 100},
    }

    pretrained_response = pretrained_predictor.predict(
        payload, custom_attributes="accept_eula=true"
    )

    print_response(payload, pretrained_response)


for i, datapoint in enumerate(test_dataset.select(range(5))):
    predict_and_print(datapoint)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please describe  what is oil and give me a list of it’s applications.

### Input:
An oil is any nonpolar chemical substance that is composed primarily of hydrocarbons and is hydrophobic (does not mix with water) & lipophilic (mixes with other oils). Oils are usually flammable and surface active. Most oils are unsaturated lipids that are liquid at room temperature.

The general definition of oil includes classes of chemical compounds that may be otherwise unrelated in structure, properties, and uses. Oils may be animal, vegetable, or petrochemical in origin, and may be volatile or non-volatile. They are used for food (e.g., olive oil), fuel (e.g., heating oil), medical purposes (e.g., mineral oil), lubrication (e.g. motor oil), and the manufacture of many types of paints, plastics, and other materials. Specially p

### 데이터 세트 S3에 업로드하기

---

준비된 데이터 세트를 미세 조정에 사용할 수 있도록 S3에 업로드합니다.

---

In [8]:
train_and_test_dataset["train"][0]

{'instruction': 'what was the british empire',
 'context': 'The British Empire was composed of the dominions, colonies, protectorates, mandates, and other territories ruled or administered by the United Kingdom and its predecessor states.',
 'response': 'The British Empire was composed of the dominions, colonies, protectorates, mandates, and other territories ruled or administered by the United Kingdom and its predecessor states. It began with the overseas possessions and trading posts established by England between the late 16th and early 18th centuries. At its height it was the largest empire in history and, for over a century, was the foremost global power. By 1913, the British Empire held sway over 412 million people, 23 per cent of the world population at the time, and by 1920, it covered 35.5 million km2 (13.7 million sq mi), 24 per cent of the Earth\'s total land area. As a result, its constitutional, legal, linguistic, and cultural legacy is widespread. At the peak of its power

In [9]:
# 훈련에 사용할 수 있도록 훈련 데이터를 로컬 파일로 저장합니다.
local_data_file = "finetuning.jsonl"
train_and_test_dataset["train"].to_json(local_data_file)

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

2066925

In [10]:
from sagemaker.s3 import S3Uploader
import sagemaker
import random

bucket = sagemaker.Session().default_bucket()

train_data_location = f"s3://{bucket}/finetuning/dolly_dataset"

S3Uploader.upload(local_data_file, train_data_location)
print(f"Training data: {train_data_location}")

Training data: s3://sagemaker-us-east-1-079002598131/finetuning/dolly_dataset


---

다음으로 훈련 작업에서 데이터를 명령어/입력 형식으로 사용하기 위한 프롬프트 템플릿을 생성합니다. (이번 예제에서는 모델을 명령어 기반 미세 조정하므로 해당 형식을 사용합니다.) 또한 배포된 엔드포인트에서 추론할 때 사용할 프롬프트 템플릿도 생성합니다.

---

In [11]:
import json

template = {
    "prompt": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{context}\n\n",
    "completion": "{response}",
}
with open("template.json", "w") as f:
    json.dump(template, f)
    
S3Uploader.upload("template.json", train_data_location)

's3://sagemaker-us-east-1-079002598131/finetuning/dolly_dataset/template.json'

In [12]:
!aws s3 ls --recursive $train_data_location

2024-01-01 21:05:51    2066925 finetuning/dolly_dataset/finetuning.jsonl
2024-01-01 21:05:52        263 finetuning/dolly_dataset/template.json


## 모델 훈련하기

---

다음으로 Dolly의 요약 데이터 세트를 사용하여 Llama 2 7B 모델을 미세 조정합니다. 미세 조정 스크립트는 [이 레포지토리](https://github.com/facebookresearch/llama-recipes/tree/main)에서 제공하는 스크립트를 기반으로 합니다. 미세 조정 스크립트에 대해 자세히 알아보려면 [5. 미세조정 방법에 대한 몇 가지 참고 사항](#5.-Few-notes-about-the-fine-tuning-method) 섹션을 확인하세요. 지원되는 하이퍼파라미터와 기본값 목록은 [3. 미세조정을 위한 지원되는 하이퍼파라미터](#3.-Supported-Hyper-parameters-for-fine-tuning) 섹션을 참조하세요.

---

In [13]:
from sagemaker.jumpstart.estimator import JumpStartEstimator

estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    instance_type="ml.g5.12xlarge",
    instance_count=2,
    environment={"accept_eula": "true"}
)

# 기본적으로 명령어 기반 조정은 비활성화 되어있습니다. 따라서 명령어 기반 조정 데이터 세트를 사용하려면
estimator.set_hyperparameters(instruction_tuned="True", 
                              epoch="5", 
                              max_input_length="1024")
estimator.fit({"training": train_data_location})

INFO:sagemaker:Creating training-job with name: meta-textgeneration-llama-2-7b-2024-01-01-21-05-52-057


2024-01-01 21:05:52 Starting - Starting the training job...
2024-01-01 21:06:19 Starting - Preparing the instances for training.......................................
2024-01-01 21:12:45 Downloading - Downloading input data..............................
2024-01-01 21:17:56 Downloading - Downloading the training image...
2024-01-01 21:18:20 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-01-01 21:18:21,967 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-01-01 21:18:22,026 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-01 21:18:22,035 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-01-01 21:18:22,036 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process gr

### 미세조정된 모델 배포하기

---

다음으로 미세 조정된 모델을 배포합니다. 이후 미세 조정된 모델과 사전 훈련된 모델의 성능을 비교합니다.

---

In [17]:
finetuned_predictor = estimator.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-2-7b-2024-01-01-22-07-44-444
INFO:sagemaker:Creating endpoint-config with name meta-textgeneration-llama-2-7b-2024-01-01-22-07-44-442
INFO:sagemaker:Creating endpoint with name meta-textgeneration-llama-2-7b-2024-01-01-22-07-44-442


-------------!

### 사전 훈련된 모델과 미세 조정된 모델 평가하기

---

다음으로 테스트 데이터를 사용하여 미세 조정된 모델의 성능을 평가하고 이를 사전 훈련된 모델과 비교합니다.

---

In [23]:
import pandas as pd
from IPython.display import display, HTML

test_dataset = train_and_test_dataset["test"]

inputs, ground_truth_responses, responses_before_finetuning, responses_after_finetuning = (
    [],
    [],
    [],
    [],
)

def predict_and_print(datapoint):
    # 명령어 기반 미세 조정을 위해 입력과 출력 사이에 특별한 키를 삽입합니다.
    input_output_demarkation_key = "\n\n### Response:\n"

    prompt = f'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{datapoint["instruction"]}\n\n### Input:\n{datapoint["context"]}\n\n',
    
    payload = {
        "inputs": prompt[0] + input_output_demarkation_key,
        "parameters": {"max_new_tokens": 100},
    }
    inputs.append(payload["inputs"])
    ground_truth_responses.append(datapoint["response"])

    pretrained_response = pretrained_predictor.predict(
        payload, custom_attributes="accept_eula=true"
    )
    responses_before_finetuning.append(pretrained_response[0]["generation"])

    finetuned_response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    responses_after_finetuning.append(finetuned_response[0]["generation"])


try:
    for i, datapoint in enumerate(test_dataset.select(range(5))):
        predict_and_print(datapoint)

    df = pd.DataFrame(
        {
            "Inputs": inputs,
            "Ground Truth": ground_truth_responses,
            "Response from non-finetuned model": responses_before_finetuning,
            "Response from fine-tuned model": responses_after_finetuning,
        }
    )
    display(HTML(df.to_html()))
except Exception as e:
    print(e)

,Inputs,Ground Truth,Response from non-finetuned model,Response from fine-tuned model
0,"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPlease describe what is oil and give me a list of it’s applications.\n\n### Input:\nAn oil is any nonpolar chemical substance that is composed primarily of hydrocarbons and is hydrophobic (does not mix with water) & lipophilic (mixes with other oils). Oils are usually flammable and surface active. Most oils are unsaturated lipids that are liquid at room temperature.\n\nThe general definition of oil includes classes of chemical compounds that may be otherwise unrelated in structure, properties, and uses. Oils may be animal, vegetable, or petrochemical in origin, and may be volatile or non-volatile. They are used for food (e.g., olive oil), fuel (e.g., heating oil), medical purposes (e.g., mineral oil), lubrication (e.g. motor oil), and the manufacture of many types of paints, plastics, and other materials. Specially prepared oils are used in some religious ceremonies and rituals as purifying agents.\n\n\n\n### Response:\n","An oil is a chemical substance that is composed primarily of hydrocarbons and may be animal, vegetable or petrochemical in origin.\nOil is used in a wide range of applications and is essential to everyday human life. These are:\nCooking - edible vegetable and animal oils are used for various purposes in cooking and food preparation\nCosmetics - most facial cleansers, lotions and hair care products contain molecules that come from mineral and vegetable oils\nFuel - crude oil is refined and converted to diesel, gasoline or jet fuel to power cars, trucks and planes\nHeating - petrochemical oil is used for heating\nPainting - oil is used as a supporting medium for paints\nLubrication - oils are used in various engineering purposes as they do not easily adhere to other substance which makes them useful as lubricants\nReligion - oil has been used throughout history as a religious medium. It is often considered a spiritually purifying agent and is used to anointing purposes\nHealth - oils holds lots of fats and medical properties, for example fish oil holds the omega-3 fatty acid which helps with inflammation and reduces fat in the bloodstream","An oil is any nonpolar chemical substance that is composed primarily of hydrocarbons and is hydrophobic (does not mix with water) & lipophilic (mixes with other oils). Oils are usually flammable and surface active. Most oils are unsaturated lipids that are liquid at room temperature.\n\nThe general definition of oil includes classes of chemical compounds that may be otherwise unrelated in structure, properties, and uses. O","Oil is made by a chemical reaction between coal and petroleum (oil).It is non-polar in nature and it is hydrophobic in nature. It is lipid of hydrocarbon compounds. It's liquid and light-colored in nature and is combustible.\nOils are used for food, fuel for cars and machines like generators, oils are used by medical.\nPaints and plastics are used in the"
1,"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the Jones-Connally Act?\n\n### Input:\nThe Jones–Connally Act was a New Deal Initiative passed by Congress in April 1934 as an extension to the Agricultural Adjustment Act. Largely in response to the great drought of 1933–1934, cattle ranchers acted against their former opposition to the commodification of cattle and appealed to the government for assistance in ridding of themselves of the millions of cattle they could no longer afford to feed or to keep alive without a loss on return.\n\n\n\n### Response:\n",The Jones–Connally Act was passed by the US Congress in April 1934. It was an extension to the Agricultural Adjustment Act. It was part of the New

### 리소스 정리하기

In [ ]:
# # 리소스 삭제
# pretrained_predictor.delete_model()
# pretrained_predictor.delete_endpoint()
# finetuned_predictor.delete_model()
# finetuned_predictor.delete_endpoint()

# 부록

### 지원되는 추론 매개변수

---
이 모델은 다음과 같은 추론 페이로드 매개변수를 지원합니다:

* **max_new_tokens:** 모델은 출력 길이(입력 컨텍스트 길이를 제외한)가 max_new_tokens에 도달할 때까지 텍스트를 생성합니다. 이 값은 반드시 양의 정수여야 합니다.
* **temperature:** 출력의 무작위성을 조절합니다. 높은 temperature 값은 낮은 확률의 단어를 포함한 출력을, 낮은 temperature 값은 높은 확률의 단어를 포함한 출력을 생성합니다. `temperature`가 0이면 탐욕적 디코딩(greedy decoding)이 수행됩니다. 이 값은 반드시 양의 실수여야 합니다.
* **top_p:** 텍스트 생성의 각 단계에서 누적 확률 `top_p`에 해당하는 가장 작은 집합의 단어들 중에서 샘플링합니다. 이 값은 0과 1 사이의 실수여야 합니다.
* **return_full_text:** True로 설정하면, 입력 텍스트가 생성된 출력 텍스트의 일부가 됩니다. 이 값은 반드시 불(boolean)이어야 하며, 기본값은 False입니다.
 
엔드포인트를 호출할 때 위에 언급된 매개변수의 하위 집합을 지정할 수 있습니다.


### 참고 사항
- `max_new_tokens`가 정의되지 않은 경우, 모델은 최대 4,000개의 전체 토큰까지 생성할 수 있습니다. 이 경우 엔드포인트 쿼리 시간 초과 오류가 발생할 수 있으므로, 가능한 경우 `max_new_tokens`를 설정하는 것이 좋습니다. 7B, 13B, 70B 모델에 대해서는 각각 `max_new_tokens`를 최대 1500, 1000, 500 이하로 설정하고, 총 토큰 수를 4,000개 이하로 유지하는 것을 권장합니다.
- 이 모델은 4,000개의 컨텍스트 길이를 지원하기 위해 배치 크기를 1로 제한하고 있습니다. 더 큰 배치 크기를 사용하는 페이로드는 추론 전에 엔드포인트 오류가 발생합니다.

---

### 미세 조정을 위해 지원되는 하이퍼파라미터

---

- epoch: 미세 조정 알고리즘이 훈련 데이터 세트를 통과하는 횟수입니다. 1보다 큰 정수여야 합니다. 기본값: 5
- learning_rate: 각 배치의 훈련 예제를 처리한 후 모델 가중치가 업데이트되는 속도입니다. 0보다 큰 양의 실수여야 합니다. 기본값: 1e-4
- instruction_tuned: 모델을 명령어로 훈련할지 여부입니다. 'True' 또는 'False' 이어야 합니다. 기본값: 'False'
- per_device_train_batch_size: 훈련을 위한 GPU 코어/CPU 당 배치 크기입니다. 양의 정수여야 합니다. 기본값: 4
- per_device_eval_batch_size: 평가를 위한 GPU 코어/CPU당 배치 크기입니다. 양의 정수여야 합니다. 기본값: 1
- max_train_samples: 디버깅 목적 또는 더 빠른 훈련을 위해 훈련 예제의 수를 이 값으로 줄입니다. -1은 모든 훈련 샘플을 사용하는 것을 의미합니다. 양의 정수 또는 -1이어야 합니다. 기본값: -1 
- max_val_samples: 디버깅 목적 또는 더 빠른 훈련을 위해 검증 예제의 수를 이 값으로 줄입니다. -1은 모든 검증 샘플을 사용하는 것을 의미합니다. 양의 정수 또는 -1이어야 합니다. 기본값: -1 
- max_input_length: 토큰화 후의 입력 시퀀스의 최대 길이입니다. 이 길이를 초과하는 시퀀스는 잘립니다. -1로 설정하면, max_input_length는 1024와 토크나이저에 의해 정의된 모델의 최대 길이 중 더 작은 값으로 설정됩니다. 양의 값으로 설정하면, max_input_length는 제공된 값과 토크나이저에 의해 정의된 모델의 최대 길이 중 더 작은 값으로 설정됩니다. 양의 정수 또는 -1이어야 합니다. 기본값: -1 
- validation_split_ratio: 검증 채널이 없는 경우, 훈련 데이터에서 훈련-검증 분할 비율입니다. 0과 1 사이여야 합니다. 기본값: 0.2 
- train_data_split_seed: 검증 데이터가 없는 경우, 알고리즘이 사용하는 훈련 데이터와 검증 데이터를 무작위로 분할하는 것을 고정합니다. 정수여야 합니다. 기본값: 0
- preprocessing_num_workers: 전처리를 위해 사용할 프로세스의 수입니다. None이면 메인 프로세스를 사용하여 전처리합니다. 기본값: "None"
- lora_r: Lora R입니다. 양의 정수여야 합니다. 기본값: 8
- lora_alpha: Lora Alpha입니다. 양의 정수여야 합니다. 기본값: 32
- lora_dropout: Lora Dropout입니다. 0과 1 사이의 양의 실수여야 합니다. 기본값: 0.05 
- int8_quantization: True로 설정하면 훈련을 위해 모델이 8비트 정밀도로 로드됩니다. 7B/13B 모델의 기본값: False. 70B 모델의 기본값: True
- enable_fsdp: True로 설정하면 완전 분할 데이터 병렬 처리(Fully Sharded Data Parallelism, FSDP)를 사용하여 훈련합니다. 7B/13B 모델의 기본값: True. 70B 모델의 기본값: False

참고 사항 1: int8_quantization은 FSDP와 함께 사용할 수 없습니다. 또한, 모든 g5 인스턴스 유형에 대해 int8_quantization = 'False'와 enable_fsdp = 'False' 설정은 CUDA 메모리 문제로 인해 지원되지 않습니다. 따라서, int8_quantization 또는 enable_fsdp 중 하나를 반드시 'True'로 설정하는 것을 권장합니다.

참고 사항 2: 모델의 크기 때문에 70B 모델은 어떤 지원 인스턴스 유형에서도 enable_fsdp = 'True' 설정으로 미세 조정할 수 없습니다.

---